In [1]:
import numpy as np
import librosa
import librosa.display
import os
from matplotlib import pyplot as plt
import random

In [2]:
# All mfcc data are contained in numpy_datas, which is a list. 
# The each entry of the list is another list consistes of [mfccs, file name]
MFCCs_DATA = "MFCCsData"
numpy_datas = []

dirlist = os.listdir(MFCCs_DATA)
for d in dirlist:
    d = os.path.join(MFCCs_DATA, d)
    datalist = os.listdir(d)
    datalist = [[np.load(os.path.join(d,x)), os.path.join(d,x)] for x in datalist]
    numpy_datas.extend(datalist)

In [3]:
numpy_datas[200][0].shape

(19, 612)

In [4]:
numpy_datas[100][0].shape

(19, 492)

Notice that the size of mfccs for each emotion file is different, need to resize the data into the same size

In [5]:
for i in range(len(numpy_datas)):
    numpy_datas[i][0] = np.resize(numpy_datas[i][0], (19,512))

In [6]:
numpy_datas[200][0].shape

(19, 512)

In [7]:
numpy_datas[100][0].shape

(19, 512)

So now all data has the same shape

Separate the data list according to the label. Change file name to correct label. [1,0,0] is angry, [0,1,0] is happy, [0,0,1] is normal.

In [8]:
# change file name to correct label
angry = []
happy = []
normal = []

for i in range(len(numpy_datas)):
    file_name = numpy_datas[i][1]
    if "angry" in file_name:
        numpy_datas[i][1] = np.array([1,0,0])
        angry.append(numpy_datas[i])
    elif "happy" in file_name:
        numpy_datas[i][1] = np.array([0,1,0])
        happy.append(numpy_datas[i])
    else:
        numpy_datas[i][1] = np.array([0,0,1])
        normal.append(numpy_datas[i])

Now split the Train-Valid-Test data with a ratio of 60%-20%-20%

In [9]:
# first shuffle the list of each emotion
random.shuffle(angry)
random.shuffle(happy)
random.shuffle(normal)

In [10]:
# then select the first 180 in each emotion as train data, first 60 in each emotion as valid data, and first 60 in each emotion as test data
train_data = angry[:int(len(angry)*0.6)] + happy[:int(len(happy)*0.6)] + normal[:int(len(normal)*0.6)]
valid_data = angry[int(len(angry)*0.6):int(len(angry)*(0.6+0.2))] + happy[int(len(happy)*0.6):int(len(happy)*(0.6+0.2))] + normal[int(len(normal)*0.6):int(len(normal)*(0.6+0.2))]
test_data = angry[int(len(angry)*(0.6+0.2)):] + happy[int(len(happy)*(0.6+0.2)):] + normal[int(len(normal)*(0.6+0.2)):]
random.shuffle(train_data)
random.shuffle(valid_data)
random.shuffle(test_data)

In [11]:
train_data

[[array([[ 1.18034569e+02,  1.23096527e+02,  1.16464035e+02, ...,
           5.96252823e+01,  2.26995964e+01, -1.28148365e+01],
         [-3.45768280e+01, -4.76046982e+01, -6.26918106e+01, ...,
          -9.18476334e+01, -9.43351822e+01, -9.72144928e+01],
         [-9.25554504e+01, -7.84741669e+01, -6.71550064e+01, ...,
           0.00000000e+00,  0.00000000e+00,  1.23317102e-02],
         ...,
         [-2.21141548e+01, -1.94214439e+01, -2.23594856e+01, ...,
          -3.70715141e+01, -2.87908535e+01, -1.78172283e+01],
         [-1.07239475e+01, -9.55793476e+00, -7.74301386e+00, ...,
          -2.28389931e+01, -1.86841125e+01, -1.85923920e+01],
         [-8.38516140e+00, -1.38220530e+01, -2.41795864e+01, ...,
          -1.03305817e+00,  7.57740736e-01,  3.63344133e-01]], dtype=float32),
  array([1, 0, 0])],
 [array([[ 10.061522  ,  -1.7643332 ,  19.841381  , ..., 118.417984  ,
          120.3461    , 131.21951   ],
         [133.13162   , 138.57407   , 139.18607   , ..., -30.56588   ,

In [12]:
# train, valid, test data are all in list consistes of [mfccs, file name]
# want to change the list to two outputs:
# X: vector of mfccs
# Y: vector of labels
def data2vector(data):
    X = []
    Y = []
    for i in range(len(data)):
        X.append(data[i][0])
        Y.append(data[i][1])
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [13]:
X_train, Y_train = data2vector(train_data)

In [14]:
print(X_train.shape)
print(Y_train.shape)

(270, 19, 512)
(270, 3)


In [15]:
X_valid, Y_valid = data2vector(valid_data)
X_test, Y_test = data2vector(test_data)

Now let's do normalization, by the value of train data

In [16]:
X_train_miu = np.average(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)

In [17]:
print(X_train_miu.shape)
print(X_train_std.shape)

(19, 512)
(19, 512)


In [18]:
X_train_norm = (X_train - X_train_miu)/X_train_std
X_valid_norm = (X_valid - X_train_miu)/X_train_std
X_test_norm = (X_test - X_train_miu)/X_train_std